# **Power Flow / Flujo de Potencia**
Este código en Python utiliza la biblioteca Pandapower para modelar y simular una red eléctrica monofásica. Primero, carga datos de entrada de diferentes componentes de la red, como buses, líneas, transformadores y cargas, desde hojas de cálculo de Excel. Luego, crea un modelo de red vacío y lo rellena con estos componentes. Adicionalmente, utiliza un perfil de demanda para ajustar dinámicamente las cargas en la red a lo largo del tiempo. Finalmente, ejecuta un flujo de potencia para cada instante temporal del perfil de demanda y almacena los resultados en hojas de cálculo para su posterior análisis.

In [1]:
#------------------------------------------------------------------------  librerias  -----------------------------------------------------------------------------------#

import numpy as np
import pandas as pd
import pandapower as pp
from pandapower.pf.runpp_3ph import runpp_3ph
from pandapower.plotting.plotly import simple_plotly
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from pandapower.plotting.plotly import pf_res_plotly
import networkx as nx
from rtree import index

In [8]:
def get_main_grid_buses(G, start_bus):
    # Inicializar una cola para el algoritmo BFS
    queue = [start_bus]
    # Inicializar un conjunto para almacenar los buses visitados
    visited = set()
    
    while queue:
        # Sacar el primer bus de la cola
        current_bus = queue.pop(0)
        # Marcar el bus actual como visitado
        visited.add(current_bus)
        # Obtener todos los vecinos del bus actual
        neighbors = list(G.neighbors(current_bus))
        # Agregar los vecinos no visitados a la cola
        for neighbor in neighbors:
            if neighbor not in visited and neighbor not in queue:
                queue.append(neighbor)
    return visited

def create_rtree(buses, net):
    rtree = index.Index()
    for bus in buses:
        if bus in net.bus_geodata.index:  # Solo considera buses con geodata
            x, y = net.bus_geodata.loc[bus, ['x', 'y']]
            rtree.insert(bus, (x, y, x, y))
    return rtree

def find_closest_section_and_bus(disconnected_sections, main_grid_buses, net):
    closest_section = None
    closest_bus = None
    closest_main_bus = None
    min_distance = float('inf')
    
    main_grid_rtree = create_rtree(main_grid_buses, net)
    
    for section in disconnected_sections:
        for disconnected_bus in section:
            if disconnected_bus not in net.bus_geodata.index:
                continue  
            
            x1, y1 = net.bus_geodata.loc[disconnected_bus, ['x', 'y']]
            epsilon = 1e-6
            nearest = list(main_grid_rtree.nearest((x1 - epsilon, y1 - epsilon, x1 + epsilon, y1 + epsilon), 1))
            if nearest:
                main_grid_bus = nearest[0]
                x2, y2 = net.bus_geodata.loc[main_grid_bus, ['x', 'y']]
                distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                if distance > 0 and distance < min_distance:
                    min_distance = distance
                    closest_section = section
                    closest_bus = disconnected_bus
                    closest_main_bus = main_grid_bus
    
    print(f'Closest section: {closest_section}, closest bus: {closest_bus}, closest main_bus: {closest_main_bus}, min_distance: {min_distance}')
    return closest_section, closest_bus, closest_main_bus, min_distance

def connect_disconnected_sections(net, G):
    disconnected_sections = list(pp.topology.connected_components(G))
    main_grid_buses = get_main_grid_buses(G, 0)  # Función ya definida para encontrar la red principal
    new_connections = []
    line_counter = 1  # Inicializar un contador para los nombres de las nuevas líneas

    while disconnected_sections:
        print(f'Number of disconnected sections: {len(disconnected_sections)}')
        closest_section, closest_bus, closest_main_bus, min_distance = find_closest_section_and_bus(disconnected_sections, main_grid_buses, net)
        if closest_bus is not None and closest_main_bus is not None:
            existing_lines = net.line[(net.line.from_bus == closest_bus) & (net.line.to_bus == closest_main_bus)]
            if existing_lines.empty:
                # Asignar un nombre único a la línea, por ejemplo: 'New Line 1', 'New Line 2', etc.
                line_name = f'New Line {line_counter} Alimentador {Alimentador}'
                new_connections.append((closest_bus, closest_main_bus, line_name))
                min_distance_km = min_distance*1000
                pp.create_line(net, from_bus=closest_bus, to_bus=closest_main_bus, length_km= 0.5, std_type="NAYY 4x50 SE", name=line_name)
                print(f'Connected bus {closest_bus} to {closest_main_bus} with {line_name}')
                line_counter += 1  # Incrementar el contador para el siguiente nombre único
            else:
                print(f'Connection already exists between bus {closest_bus} and {closest_main_bus}')

            G = pp.topology.create_nxgraph(net, multi=False)
            main_grid_buses = get_main_grid_buses(G, 0)
            disconnected_sections = list(pp.topology.connected_components(G))
            disconnected_sections = [s for s in disconnected_sections if s != closest_section]
        else:
            print('No closest bus or main bus found, exiting loop')
            break

    return net, new_connections

In [4]:
# Función para crear la red eléctrica
def crear_red_electrica(Filename,Alimentador):
    # Definición de DataFrames para diferentes componentes de la red
    hv_bus = pd.read_excel(Filename, sheet_name="buses", index_col=[0])  # Buses

    hv_lines = pd.read_excel(Filename, sheet_name="lineas")  # Líneas

    hv_trafo = pd.read_excel(Filename, sheet_name="transformadores")  # Transformadores

    hv_grid = pd.read_excel(Filename, sheet_name="grid")  # Red externa

    Catalogo_conductor = pd.read_excel(Filename, sheet_name="catalogo_conductor")  # Catálogo de conductores

    Banco_cond = pd.read_excel(Filename, sheet_name="Banco_cond")  # Bancos de condensadores

    bus_indices_to_drop = hv_bus[hv_bus['vn_kv'].isin([0.22, 0.38]) | hv_bus['vn_kv'].isna()].index

    # Eliminar buses de baja tensión :
    hv_bus = hv_bus.drop(bus_indices_to_drop)

    # Identificar los bus_id que están presentes en la columna bus_hv de hv_trafo y también en hv_bus
    valid_bus_ids_hv_trafo = set(hv_trafo['bus_hv'].unique()) & set(hv_bus.index)

    # Obtener los índices de los buses válidos desde el DataFrame de buses
    valid_bus_indices = set(hv_bus.index)

    # Identificar las líneas que tienen buses de conexión no válidos
    # Comprobamos tanto el bus de origen ('from_bus') como el bus de destino ('to_bus')
    invalid_lines = hv_lines[~(hv_lines['from_bus'].isin(valid_bus_indices) & hv_lines['to_bus'].isin(valid_bus_indices))]

    # Eliminar las líneas no válidas del DataFrame hv_lines
    hv_lines = hv_lines.drop(invalid_lines.index)

    # Crear una red vacía
    net = pp.create_empty_network()

    # Definir buses
    for _, bus_df in hv_bus.iterrows():
        pp.create_bus(net, name=bus_df.name, vn_kv=bus_df.vn_kv, type="n", 
                      in_service=True, geodata=(bus_df.lat, bus_df.long))

    # Definir catálogo de conductores
    for _, conductor_df in Catalogo_conductor.iterrows():
        std_type = {
            "max_i_ka": conductor_df.max_i_ka,
            "r_ohm_per_km": conductor_df.r_ohm_per_km,
            "x_ohm_per_km": conductor_df.x_ohm_per_km,
            "c_nf_per_km": conductor_df.c_nf_per_km,
            "g_us_per_km": conductor_df.g_us_per_km
        }
        pp.create_std_type(net, std_type, name=conductor_df.Conductor_id, element="line")

    # Definir líneas
    for _, line_df in hv_lines.iterrows():
        from_bus = pp.get_element_index(net, "bus", line_df.from_bus)
        to_bus = pp.get_element_index(net, "bus", line_df.to_bus)
        pp.create_line(net, from_bus, to_bus, length_km=line_df.length_km,
                       std_type=line_df.std_type, name=line_df.line_name, 
                       in_service=True, df=1.0, parallel=1)

    # Definir red externa
    for _, grid_df in hv_grid.iterrows():
        bus_idx = pp.get_element_index(net, "bus", grid_df.bus)
        pp.create_ext_grid(net, bus_idx, vm_pu=grid_df.vm_pu,
                           va_degree=grid_df.va_degree, name=grid_df.grid_name, in_service=True)

    # Definir banco de condensadores
    for _, bank_df in Banco_cond.iterrows():
        bus_idx = pp.get_element_index(net, "bus", bank_df.bus)
        pp.create_shunt_as_capacitor(
            net, bus_idx, name=bank_df.bank_name, q_mvar=bank_df.q_mvar,
            loss_factor=bank_df.loss_factor, vn_kv=bank_df.vn_kv, in_service=True)
    
    pp.diagnostic(net, report_style='detailed', warnings_only=True, return_result_dict=True, overload_scaling_factor=0.001, min_r_ohm=0.001,
                           min_x_ohm=0.001, min_r_pu=1e-05, min_x_pu=1e-05, nom_voltage_tolerance=0.3, numba_tolerance=1e-05)
    
    G = pp.topology.create_nxgraph(net, multi=False)

    # Obtener el conjunto de buses en la red principal
    main_grid_buses = get_main_grid_buses(G, 0)
    
    net.bus_geodata = net.bus_geodata.dropna(subset=['x', 'y'])

    # Llamar a la función para conectar las secciones desconectadas
    net, new_connections = connect_disconnected_sections(net, G)

     # Ejecutar diagnóstico
    result_dict = pp.diagnostic(net, report_style='detailed', warnings_only=False, return_result_dict=True, overload_scaling_factor=0.001, min_r_ohm=0.001,
                           min_x_ohm=0.001, min_r_pu=1e-05, min_x_pu=1e-05, nom_voltage_tolerance=0.3, numba_tolerance=1e-05)
    # Inicializar listas para almacenar elementos desconectados
    disconnected_buses = []
    disconnected_lines = []
    disconnected_transformers = []
    disconnected_loads = []
    # Extraer y almacenar elementos desconectados
    for section in result_dict.get('disconnected_elements', []):
        disconnected_buses.extend(section.get('buses', []))
        disconnected_lines.extend(section.get('lines', []))
        disconnected_transformers.extend(section.get('trafos', []))
        disconnected_loads.extend(section.get('loads', []))

    # Eliminar duplicados (si los hay)
    disconnected_buses = list(set(disconnected_buses))
    disconnected_lines = list(set(disconnected_lines))
    disconnected_transformers = list(set(disconnected_transformers))
    disconnected_loads = list(set(disconnected_loads))
        # Eliminación de elementos desconectados
    for bus in disconnected_buses:
        net.bus.drop(bus, inplace=True)

    for line in disconnected_lines:
        net.line.drop(line, inplace=True)

    for trafo in disconnected_transformers:
        net.trafo.drop(trafo, inplace=True)

    for load in disconnected_loads:
        net.load.drop(load, inplace=True)

    bus_index_to_name = net.bus['name'].to_dict()
    
    # Agrega nuevas columnas 'from_bus_name' y 'to_bus_name' al DataFrame net.line
    net.line['from_bus_name'] = net.line['from_bus'].map(bus_index_to_name)
    net.line['to_bus_name'] = net.line['to_bus'].map(bus_index_to_name)
    
    net.trafo['hv_bus_name'] = net.trafo.hv_bus.map(bus_index_to_name)
    net.trafo['lv_bus_name'] = net.trafo.lv_bus.map(bus_index_to_name)
    net.ext_grid['bus_name'] = net.ext_grid.bus.map(bus_index_to_name)
    net.shunt['bus_name'] = net.shunt.bus.map(bus_index_to_name)
    bus_data = pd.merge(net.bus, net.bus_geodata, how='left', left_index=True, right_index=True)
    bus_data['alimentador'] = Alimentador
    net.bus_geodata['Alimentador'] = Alimentador
    net.line['alimentador'] = Alimentador
    net.trafo['alimentador'] = Alimentador
    net.ext_grid['alimentador'] = Alimentador
    net.load['alimentador'] = Alimentador
    net.shunt['alimentador'] = Alimentador
    Catalogo_conductor['alimentador'] = Alimentador
    with pd.ExcelWriter(f'Output\Alimentador{Alimentador}_corregido.xlsx') as writer:
        bus_data.to_excel(writer, sheet_name='Bus Data', index =False)
        net.line.to_excel(writer, sheet_name='Line Data')
        net.trafo.to_excel(writer, sheet_name='Transformer Data')
        net.ext_grid.to_excel(writer, sheet_name='External Grid Data')
        net.load.to_excel(writer, sheet_name='Load Data')
        net.shunt.to_excel(writer, sheet_name='Shunt Data')
        Catalogo_conductor.to_excel(writer, sheet_name='std_type Data')

    return net

In [6]:
Filename = "Input\Alimentadores\Alimentador677.xlsx"  # Datos de la red
Alimentador = 677
crear_red_electrica(Filename,Alimentador)

Filename = "Input\Alimentadores\Alimentador675.xlsx"  # Datos de la red
Alimentador = 675
crear_red_electrica(Filename,Alimentador)

Filename = "Input\Alimentadores\Alimentador665.xlsx"  # Datos de la red
Alimentador = 665
crear_red_electrica(Filename,Alimentador)

Filename = "Input\Alimentadores\Alimentador678.xlsx"  # Datos de la red
Alimentador = 678
crear_red_electrica(Filename,Alimentador)



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [0, 171, 478, 479, 644, 658, 664, 1382, 1383, 1604, 1718, 1928, 1932, 1934, 1961, 2164, 2171, 2368]
lines: [1090, 1094, 1097, 1129, 2058, 1069, 1678, 2061, 1105, 1106, 1107, 2065, 1077, 2067, 2070, 2073, 1087]
Disconnected section found, consisting of the following elements:
buses: [1, 15, 205, 265, 370, 386, 1095, 1265, 1302, 1308, 1310, 1311, 1318, 1321, 1325, 1350, 1351, 1473, 1495, 1503, 1520, 1521, 1530, 1531, 1534, 1682, 1692, 1782, 1938, 2008, 2022, 2024, 2178, 2385]
lines: [777, 526, 783, 914, 787, 916, 792, 538, 926, 799, 543, 548, 804, 945, 562, 820, 693, 948, 830, 834, 585, 588, 977, 599, 986, 732, 989, 991, 624, 752, 882, 758, 766]
Disconnected section found, consisting of the following elements:
buses: [2, 99, 1846, 2101]
lines: [2005, 1658, 1989]
Disconnected section found, co

Number of disconnected sections: 226
Closest section: {1266, 1660, 1653}, closest bus: 1653, closest main_bus: 1718, min_distance: 0.00012071491209461047
Connected bus 1653 to 1718 with New Line 1 Alimentador 675
Number of disconnected sections: 225
Closest section: {1690, 2163, 1166, 1671}, closest bus: 1671, closest main_bus: 2164, min_distance: 0.0002693124950771515
Connected bus 1671 to 2164 with New Line 2 Alimentador 675
Number of disconnected sections: 224
Closest section: {1387}, closest bus: 1387, closest main_bus: 1382, min_distance: 0.0002785937005789811
Connected bus 1387 to 1382 with New Line 3 Alimentador 675
Number of disconnected sections: 223
Closest section: {1446, 967, 1416, 1433, 1420, 1424, 1238, 57, 701}, closest bus: 57, closest main_bus: 1387, min_distance: 0.0002627677681880076
Connected bus 57 to 1387 with New Line 4 Alimentador 675
Number of disconnected sections: 222
Closest section: {2377}, closest bus: 2377, closest main_bus: 1446, min_distance: 0.00011776



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [1323, 1500, 1665, 1754, 2314]
lines: [58, 211, 125, 227]
Disconnected section found, consisting of the following elements:
buses: [11, 1362]
lines: [680]
Disconnected section found, consisting of the following elements:
buses: [14, 117]
lines: [655]
Disconnected section found, consisting of the following elements:
buses: [19, 2224]
lines: [670]
Disconnected section found, consisting of the following elements:
buses: [23, 37]
lines: [2212]
Disconnected section found, consisting of the following elements:
buses: [31, 32]
lines: [1721]
Disconnected section found, consisting of the following elements:
buses: [33]
Disconnected section found, consisting of the following elements:
buses: [34, 704]
lines: [636]
Disconnected section found, consisting of the following elements:
buses: [3

Number of disconnected sections: 70
Closest section: {43, 47}, closest bus: 47, closest main_bus: 45, min_distance: 9.566801973274185e-05
Connected bus 47 to 45 with New Line 1 Alimentador 665
Number of disconnected sections: 69
Closest section: {568, 300, 14, 543}, closest bus: 568, closest main_bus: 0, min_distance: 9.592085278813844e-05
Connected bus 568 to 0 with New Line 2 Alimentador 665
Number of disconnected sections: 68
Closest section: {560, 553}, closest bus: 553, closest main_bus: 568, min_distance: 9.771985468810137e-05
Connected bus 553 to 568 with New Line 3 Alimentador 665
Number of disconnected sections: 67
Closest section: {80, 655}, closest bus: 655, closest main_bus: 88, min_distance: 0.00010040602571594855
Connected bus 655 to 88 with New Line 4 Alimentador 665
Number of disconnected sections: 66
Closest section: {581, 646, 615, 650, 555, 749, 144, 147, 86, 534, 150, 637}, closest bus: 144, closest main_bus: 96, min_distance: 0.0002874393327323538
Connected bus 144



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [42, 751]
lines: [122]
Disconnected section found, consisting of the following elements:
buses: [752]

SUMMARY: 4 disconnected element(s) found.

 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------

_____________ END OF PANDAPOWER DIAGNOSTIC _____________ 


_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [0, 11, 97, 132, 362, 363, 378, 382, 383, 486, 1151]
lines: [65, 97, 69, 40, 41, 16, 80, 18, 88, 94]
Disconnected section found, consisting of the following elements:
buses: [1, 18, 1329]
lines: [589, 318]
Disconnected section foun

Number of disconnected sections: 137
Closest section: {545, 194, 324, 8, 424, 1246, 1119}, closest bus: 1246, closest main_bus: 11, min_distance: 0.0007990721244089153
Connected bus 1246 to 11 with New Line 1 Alimentador 678
Number of disconnected sections: 136
Closest section: {1045, 1052, 1053, 1057, 1077, 1084, 576, 1102, 1106, 1107, 596, 1110, 608, 610, 1124, 1129, 621, 1135, 1136, 119, 1149, 1150, 640, 651, 1163, 663, 152, 667, 155, 677, 1192, 1193, 691, 1204, 694, 1207, 195, 200, 715, 1227, 1230, 722, 1242, 1243, 223, 1249, 1256, 233, 746, 1263, 753, 1277, 1294, 273, 785, 290, 299, 811, 317, 837, 326, 327, 859, 350, 862, 872, 879, 880, 373, 901, 910, 403, 923, 926, 928, 930, 968, 459, 974, 981, 489, 1010, 505, 1023}, closest bus: 317, closest main_bus: 1119, min_distance: 0.0002590577734853064
Connected bus 317 to 1119 with New Line 2 Alimentador 678
Number of disconnected sections: 135
Closest section: {1068, 944, 1078, 760, 1017, 860}, closest bus: 860, closest main_bus: 928, m



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [3, 7]
lines: [553]
Disconnected section found, consisting of the following elements:
buses: [225, 517]
lines: [460]
Disconnected section found, consisting of the following elements:
buses: [170, 205, 243, 255, 257, 263, 270, 276, 287, 294, 301, 307, 323, 348, 366, 387, 422, 438, 474, 479, 620, 737, 748, 787, 812, 829, 858, 881, 900, 922, 924, 932, 969, 1031, 1036, 1056, 1083, 1101, 1128, 1162, 1171, 1172, 1224, 1237, 1278]
lines: [640, 645, 646, 650, 651, 654, 657, 658, 665, 667, 668, 671, 683, 685, 687, 689, 690, 692, 696, 699, 701, 702, 706, 710, 711, 716, 719, 720, 726, 728, 729, 733, 736, 741, 742, 744, 746, 747, 748, 749, 751, 755, 634, 635]
Disconnected section found, consisting of the following elements:
buses: [9, 40, 46]
lines: [1241, 1242]
Disconnected section found, 

This pandapower network includes the following parameter tables:
   - bus (1290 element)
   - ext_grid (1 elements)
   - line (1289 element)
   - bus_geodata (1385 element)
 and the following results tables:
   - res_bus (1385 element)
   - res_line (1372 element)
   - res_ext_grid (1 elements)